In [ ]:
import pandas as pd
import numpy as np
import sys

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), 'Physical GPUs,', len(logical_gpus), 'Logical GPUs')
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# dataset = "savee"
# x_train = np.load(f"./saved_datasets/{dataset}_x_train.npy")
# y_train = np.load(f"./saved_datasets/{dataset}_y_train.npy")
# x_test = np.load(f"./saved_datasets/{dataset}_x_test.npy")
# y_test = np.load(f"./saved_datasets/{dataset}_y_test.npy")
# x_val = np.load(f"./saved_datasets/{dataset}_x_val.npy")
# y_val = np.load(f"./saved_datasets/{dataset}_y_val.npy")

In [ ]:
# Do użycia w przypadku trenowania nowych modeli

# model=Sequential()
# model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

# model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

# model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
# model.add(Dropout(0.2))

# model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

# model.add(Flatten())

# model.add(Dense(units=32, activation='relu')) # Wariant 1

# model.add(Dense(units=128, activation='relu')) # Wariant 2
# model.add(Dense(units=128, activation='relu'))


# model.add(Dropout(0.3))
# model.add(Dense(units=y_train.shape[1], activation='softmax'))
# model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

# model.summary()

In [ ]:
# Do użycia w przypadku dotrenowywania modeli

# model_name = "savee-tess-ravdess"
# dataset = "cremad"

# x_train = np.load(f"./saved_datasets/{dataset}_x_train.npy")
# y_train = np.load(f"./saved_datasets/{dataset}_y_train.npy")
# x_test = np.load(f"./saved_datasets/{dataset}_x_test.npy")
# y_test = np.load(f"./saved_datasets/{dataset}_y_test.npy")
# x_val = np.load(f"./saved_datasets/{dataset}_x_val.npy")
# y_val = np.load(f"./saved_datasets/{dataset}_y_val.npy")

# print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape)

# model = tf.keras.models.load_model(f'./models/{model_name}_alt_model')

In [ ]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
history=model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_val, y_val), callbacks=[rlrp])

In [ ]:
print("Dokładność modelu na zbiorze testowym: " , model.evaluate(x_test,y_test)[1]*100 , "%")

epochs = [i for i in range(50)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(epochs , train_loss , label = 'Wartość funkcji straty w na zbiorze treningowym')
ax[0].plot(epochs , test_loss , label = 'Wartość funkcji straty w na zbiorze walidacyjnym')
ax[0].set_title('Wartość funkcji straty na zbiorach treningowym i testowym')
ax[0].legend()
ax[0].set_xlabel("Epoki")

ax[1].plot(epochs , train_acc , label = 'Dokładność na zbiorze treningowym')
ax[1].plot(epochs , test_acc , label = 'Dokładność na zbiorze walidacyjnym')
ax[1].set_title('Dokładność na zbiorach treningowym i testowym')
ax[1].legend()
ax[1].set_xlabel("Epoki")
# plt.savefig(f"./plots/{model_name}-{dataset}-model_alt_{dataset}-train_history")
# plt.savefig(f"./plots/{dataset}-model_alt-train_history")
plt.show()

In [ ]:
Features = pd.read_csv(f"./features/{dataset}_features.csv")
Y = Features['labels'].values
del Features

In [ ]:
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

pred_test = model.predict(x_test)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

In [ ]:
df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(10)